In [1]:
import pandas as pd
import numpy as np
import re
from utils import cleaning

In [2]:
df = pd.read_csv("dataset/world_energy_consumption.csv")

In [3]:
df.head()

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1900,AFG,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,AFG,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,AFG,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,AFG,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,AFG,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp',
       'biofuel_cons_change_pct', 'biofuel_cons_change_twh',
       'biofuel_cons_per_capita', 'biofuel_consumption',
       'biofuel_elec_per_capita',
       ...
       'solar_share_elec', 'solar_share_energy', 'wind_cons_change_pct',
       'wind_cons_change_twh', 'wind_consumption', 'wind_elec_per_capita',
       'wind_electricity', 'wind_energy_per_capita', 'wind_share_elec',
       'wind_share_energy'],
      dtype='object', length=128)

In [5]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22343 entries, 0 to 22342
Data columns (total 128 columns):
 #    Column                                        Dtype  
---   ------                                        -----  
 0    country                                       object 
 1    year                                          int64  
 2    iso_code                                      object 
 3    population                                    float64
 4    gdp                                           float64
 5    biofuel_cons_change_pct                       float64
 6    biofuel_cons_change_twh                       float64
 7    biofuel_cons_per_capita                       float64
 8    biofuel_consumption                           float64
 9    biofuel_elec_per_capita                       float64
 10   biofuel_electricity                           float64
 11   biofuel_share_elec                            float64
 12   biofuel_share_energy                        

In [6]:
# Dump value counts
cleaning.dump_value_counts('data_expl/', df)

Try to subdivide the dataset in other sub-dataset.
1. One will be the ones with all the primary energy productions, consumption,
   shares etc.
2. Other will be with overall measures such as overall energy production
<br><br>
After that, an EDA will be performed, it will also be easier to understand the <br> 
dataset's behaviour (missing data per country, missing data per energy src etc.) <br>
These sub-dataset can be indexed by the identifiers that reports general id like <br>
country, population and gdp. 

Which kind of data is available for energy src?

In [7]:
# Categorize columns
pure_energy_src = ['biofuel', 'coal', 'gas', 'hydro', 'nuclear', 'oil', 'solar', 'wind']
aggragated_energy_src = ['fossil', 'low-carbon', 'other-renewables', 'other-renewables-exc-biofuel', 'renewables']
overall_energy_measures = ['energy_cons_change_pct', 'energy_cons_change_pct', 
                    'carbon_intensity_elec', 'electricity_generation', 
                    'fossil_electricity', 'renewables_electricity',
                    'energy_per_gdp', 'energy_per_capita', 
                    'per_capita_electricity', 'primary_energy_consumption']
identifiers = ['iso_code', 'country', 'year', 'population', 'gdp']

In [8]:
# Std consumption and production names:
# production --> prod
# consumption --> cons
# electricity --> elec
df_std = df.copy()

df_std.columns = df.columns.str.replace('production', 'prod').str.replace('electricity', 'elec').str.replace('consumption', 'cons')
df_std.columns = df_std.columns.str.replace('renewable', 'renewables')
df_std.columns = df_std.columns.str.replace('renewabless', 'renewables')
df_std.columns = df_std.columns.str.replace('low_carbon', 'low-carbon')
df_std.columns = df_std.columns.str.replace('other_renewables', 'other-renewables')
df_std.columns = df_std.columns.str.replace('other_renewables_exc_biofuel', 'other-renewables-exc-biofuel')

# if after a energy src "energy" is present, it is an alias for primary energy consumption "cons" and could be substitued
old_cols = list(df_std.columns)
new_cols = old_cols
for i in range(len(df.columns)):
    # Check that col refer to an energy src and 'energy' keyword is in 2nd position
    splitted_col = old_cols[i].split('_')
    if splitted_col[0] in pure_energy_src and splitted_col[1] == 'energy':
        splitted_col[1] = 'cons'
        new_cols[i] = '_'.join(splitted_col)

df_std.columns = new_cols

### Template column
energysrc_measure_opt_measure_opt_unit

For each pure energy src we have the following:
1. Consumption data **cons**
   For **cons** data then we have
   1. Primary fuel consumption if only **cons** in terawatt-hours
   2. **change** --> representing annual change in energy src consumption
      1. **change** can be percentatage with **pct**
      2. **change** can be in terawatt-hours with **twh**
   3. **per_capita** if measured in terwatt-hours per capita
2. Electricity consumption from the energy src with **elec**:
   1. Pure electricity consumption if only **elec** in terawatt-hours
   2. **per_capita** if electricity is measured per capita
3. **share** when accounting for the share among the other energy srcs:
   1. could be **elec** referring to electricity consuption;
   2. could be **energy** when accounting for energy srcs
4. **prod** accounts for prod (elec+thermal?) using the energy source:
   1. if no additional keywords, is the overall production in terawatt hours;
   2. with **per_capita** accounts with the production per capita in twh;
   3. with **change** keyword accounts for the annual change in production:
      1. could be percentage if **pct**
      2. could be in twh if **twh**

In [9]:
# What type of measures are available for pure energy srcs
measures_type = [([col for col in df_std.columns if energy == col.split('_')[0]]) for energy in pure_energy_src]
measures_per_energy = {pure_energy_src[i]: measures_type[i] for i in range(len(measures_type))}
measures_per_energy

{'biofuel': ['biofuel_cons_change_pct',
  'biofuel_cons_change_twh',
  'biofuel_cons_per_capita',
  'biofuel_cons',
  'biofuel_elec_per_capita',
  'biofuel_elec',
  'biofuel_share_elec',
  'biofuel_share_energy'],
 'coal': ['coal_cons_change_pct',
  'coal_cons_change_twh',
  'coal_cons_per_capita',
  'coal_cons',
  'coal_elec_per_capita',
  'coal_elec',
  'coal_prod_change_pct',
  'coal_prod_change_twh',
  'coal_prod_per_capita',
  'coal_prod',
  'coal_share_elec',
  'coal_share_energy'],
 'gas': ['gas_cons_change_pct',
  'gas_cons_change_twh',
  'gas_cons',
  'gas_elec_per_capita',
  'gas_elec',
  'gas_cons_per_capita',
  'gas_prod_change_pct',
  'gas_prod_change_twh',
  'gas_prod_per_capita',
  'gas_prod',
  'gas_share_elec',
  'gas_share_energy'],
 'hydro': ['hydro_cons_change_pct',
  'hydro_cons_change_twh',
  'hydro_cons',
  'hydro_elec_per_capita',
  'hydro_elec',
  'hydro_cons_per_capita',
  'hydro_share_elec',
  'hydro_share_energy'],
 'nuclear': ['nuclear_cons_change_pct',
  '

In [10]:
# How many measures are available for pure energy srcs
n_measures = [len([col for col in df_std.columns if energy == col.split('_')[0]]) for energy in pure_energy_src]
measures_per_energy = {pure_energy_src[i]: n_measures[i] for i in range(len(n_measures))}
measures_per_energy

{'biofuel': 8,
 'coal': 12,
 'gas': 12,
 'hydro': 8,
 'nuclear': 8,
 'oil': 12,
 'solar': 8,
 'wind': 8}

In [11]:
# What type of measures are available for aggregated energy srcs
measures_type = [([col for col in df_std.columns if energy == col.split('_')[0]]) for energy in aggragated_energy_src]
measures_per_energy = {aggragated_energy_src[i]: measures_type[i] for i in range(len(measures_type))}
measures_per_energy

{'fossil': ['fossil_cons_change_pct',
  'fossil_cons_change_twh',
  'fossil_elec_per_capita',
  'fossil_elec',
  'fossil_energy_per_capita',
  'fossil_fuel_cons',
  'fossil_share_elec',
  'fossil_share_energy'],
 'low-carbon': ['low-carbon_cons_change_pct',
  'low-carbon_cons_change_twh',
  'low-carbon_cons',
  'low-carbon_elec_per_capita',
  'low-carbon_elec',
  'low-carbon_energy_per_capita',
  'low-carbon_share_elec',
  'low-carbon_share_energy'],
 'other-renewables': ['other-renewables_cons',
  'other-renewables_elec',
  'other-renewables_exc_biofuel_elec',
  'other-renewables_cons_change_pct',
  'other-renewables_cons_change_twh',
  'other-renewables_elec_per_capita',
  'other-renewables_elec_per_capita_exc_biofuel',
  'other-renewables_energy_per_capita',
  'other-renewables_share_elec',
  'other-renewables_share_elec_exc_biofuel',
  'other-renewables_share_energy'],
 'other-renewables-exc-biofuel': [],
 'renewables': ['renewables_cons_change_pct',
  'renewables_cons_change_twh',

In [13]:
n_measures = [len([col for col in df_std.columns if energy == col.split('_')[0]]) for energy in aggragated_energy_src]
measures_per_energy = {aggragated_energy_src[i]: n_measures[i] for i in range(len(n_measures))}
measures_per_energy

{'fossil': 8,
 'low-carbon': 8,
 'other-renewables': 11,
 'other-renewables-exc-biofuel': 0,
 'renewables': 8}

In [12]:
df_std.columns[30:]

Index(['energy_per_capita', 'energy_per_gdp', 'fossil_cons_change_pct',
       'fossil_cons_change_twh', 'fossil_elec_per_capita', 'fossil_elec',
       'fossil_energy_per_capita', 'fossil_fuel_cons', 'fossil_share_elec',
       'fossil_share_energy', 'gas_cons_change_pct', 'gas_cons_change_twh',
       'gas_cons', 'gas_elec_per_capita', 'gas_elec', 'gas_cons_per_capita',
       'gas_prod_change_pct', 'gas_prod_change_twh', 'gas_prod_per_capita',
       'gas_prod', 'gas_share_elec', 'gas_share_energy',
       'greenhouse_gas_emissions', 'hydro_cons_change_pct',
       'hydro_cons_change_twh', 'hydro_cons', 'hydro_elec_per_capita',
       'hydro_elec', 'hydro_cons_per_capita', 'hydro_share_elec',
       'hydro_share_energy', 'low-carbon_cons_change_pct',
       'low-carbon_cons_change_twh', 'low-carbon_cons',
       'low-carbon_elec_per_capita', 'low-carbon_elec',
       'low-carbon_energy_per_capita', 'low-carbon_share_elec',
       'low-carbon_share_energy', 'net_elec_imports',
      